# The Treasure And Shoe Archive from Drehem

The Treasure and Shoe archive from Drehem (ancient Puzriš-Dagan) is a relatively small archive of administrative texts that date to the 21st century BCE and deal with valuable objects (made of metals and stone) and leather objects (shoes and belts, etc.. The archive was studied in detail by Paola Paoletti in her book *Der König und sein Kreis* (2012). The present notebook essentially takes the texts edited and studied by Paoletti, assigning roles to personal names in each text. This data is then used to study the network of persons involved in this archive. The network is drawn in an interactive plot.

This notebook goes through several steps: 
* Data acquisition
* Data cleaning
* Defining Nodes
* Assigning roles
* Defining edges
* Creating a Network (using networkx)
* Plotting the network (using bokeh)

# Import the Necessary Modules
Here we import the modules necessary for data acquisition and data cleaning and for assignment of roles and edges.

In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) # this suppresses a warning about pandas from tqdmimport pandas as pd
import os
import sys
import zipfile
import json
from tqdm.auto import tqdm
tqdm.pandas()
from ipywidgets import interact
import re
import pickle
util_dir = os.path.abspath('../utils')
sys.path.append(util_dir)
from utils import *

# The Corpus

A list of P numbers (text ID numbers) of texts that belong to the Treasure/Shoe archive is available in the directory `csv`, kindly made available by Paolo Paoletti. This list contains some additional information, for instance where a text was published, and to which sub-archive it belongs. The latter information may be used later on to separate the Treasure texts from the leather texts (or to color those differently in the plot).

One of the texts studied in her book still remains unpublished and is therefore not included (this is U.30117). Several additional Treasury texts were published by [Nawalla Al-Mutawalli and Walther Sallaberger 2018](https://doi-org.libproxy.berkeley.edu/10.1515/za-2017-0101) and one additional leather text is found in [RSO 83 (2010), 345 text 14](http://oracc.org/epsd2/admin/ur3/P433582). Another leather text was identified among the photographs of texts from the Louvre ([AO 11393](http://oracc.org/epsd2/admin/ur3/P493329)) made available on [CDLI](http://cli.ucla.edu/P493329).

The corpus used here is, therefore, not exactly identical to what was used in Paoletti's book - but it is very close. At the moment of writing the corpus that is being used includes 288 documents.

In [2]:
file = 'csv/treasury_shoes2.txt'
tr_df = pd.read_csv(file, encoding='utf8')
ids = list(tr_df['id_text'])
ids.sort()

# Downloading the Corpus
When running this notebook for the first time (or after a major update of the Ur III corpus on [epsd2](http://oracc.org/epsd2/admin/ur3) it is necessary to download the entire Ur III corpus in JSON format. If you already have the file `epsd2-admin-ur3.zip` in the `jsonzip` directory you may comment out the command that does the download, by placing a hashmark (#) in front of it as follows: 

``` python
#oracc_download([project])
```

Remove the hashmark and run the cell again to download a fresh copy (this is a large file and may take a while, depending on your computer and your connection).

In [3]:
project = 'epsd2/admin/ur3'
#oracc_download([project]);

# The Parsejson function

The `parsejson()` function used here differs in one way from the ones demonstrated in Chapter 2.1. The code looks for a `key` `ftype` (meaning field type) among the lemmatization data. The only value that `ftype` may take in this corpus is `yn`, indicating that a word is part of a year name. By including that data we may easily exclude year names from our analysis. We will, of course, consider the dating of each texts (that is part of the metadata). The vocabulary of the year names, however, is irrelevant for understanding the transactions and connections described in this corpus.

In [4]:
def parsejson(text):
    for JSONobject in text["cdl"]:
        if "cdl" in JSONobject: 
            parsejson(JSONobject)
        if "label" in JSONobject:
            meta_d["label"] = JSONobject['label']
        if "f" in JSONobject:
            lemma = JSONobject["f"]
            if "ftype" in JSONobject:
                lemma['ftype'] = JSONobject['ftype'] # this picks up YN for year name
            lemma["id_word"] = JSONobject["ref"]
            lemma['label'] = meta_d["label"]
            lemma["id_text"] = meta_d["id_text"]
            lemm_l.append(lemma)
        if "strict" in JSONobject and JSONobject["strict"] == "1":
            lemma = {key: JSONobject[key] for key in dollar_keys}
            lemma["id_word"] = JSONobject["ref"]
            lemma["id_text"] = meta_d["id_text"]
            lemm_l.append(lemma)
    return

# Call the Parseron Treasury Texts

The variable `ids`, created above, holds all the P numbers of the documents we are interested in. This list is used to define the list of files to be extracted from the `zip` file and to be sent to the parser. Otherwise, the code below is identical to the code in Chapter 2.1.

In [5]:
lemm_l = []
meta_d = {"label": None, "id_text": None}
dollar_keys = ["extent", "scope", "state"]
file = f"jsonzip/{project.replace('/', '-')}.zip"
try:
    z = zipfile.ZipFile(file) 
except:
    print(f"{file} does not exist or is not a proper ZIP file")
files = z.namelist() # list of all the files in the ZIP file
files = [name for name in files if name[-12:-5] in ids] # select only those of the treasury/leather archive
for filename in tqdm(files, desc = project):
    id_text = project + filename[-13:-5] 
    meta_d["id_text"] = id_text
    try:
        st = z.read(filename).decode('utf-8')
        data_json = json.loads(st)           
        parsejson(data_json)
    except:
        print(f'{id_text} is not available or not complete')
z.close()

epsd2/admin/ur3/P143238 is not available or not complete



# DataFrame

The `parsejson()` function has filled the list of lists `lemm_l` with data. This list  is read into a DataFrame for further manipulation. We remove commas and spaces from Guide Words and simplify the field `id_text` to read "P433582" instead of "epsd2/admin/ur3/P433582" (all documents derive from [epsd2/admin/ur3](http://oracc.org/epsd2/admin/ur3)).

Finally the code adds a field `id_line` in order to keep track of the lines in a document. Since lines are meaningful units in Sumerian texts, this will be an important tool. 

All of these steps are identical (or very similar) to steps explained in Chapter 2.1.

In [6]:
words = pd.DataFrame(lemm_l).fillna('')
keep = ['extent', 'scope', 'state', 'id_word', 'id_text', 'form', 'cf', 'gw', 'pos', 'ftype', 'label']
words = words[keep]
words['gw'] = words['gw'].replace([' ', ','], ['', ''], regex=True)
words['id_text'] = [i[-7:] for i in words['id_text']]
words['id_line'] = [int(i.split('.')[1]) for i in words['id_word']]

# Creating a Lemma Field

The `lemma` field strings Citation Form, Guide Word, and Part of Speech together and is dealt with in the same way as in Chapter 2.1 Words that remain unlemmatized (e.g. because of illegible or broken signs) are represented by their `form` and numbers receive the POS 'NU'. 

In the DataFrame (physical) breaks and text divisions (marked by horizontal rulings on the tablet, blank lines, and the like) are preserved in the field `state`. Such demarcations have their own entry in the DataFrame and receive their own line ID, but they do not have data in fields such as `form`. In order to keep track of breaks we will define two different types: logical and physical breaks. The keywords break_physical and break_logical are included in the `lemma` column. 

In [7]:
physical_break = ['illegible', 'traces', 'missing', 'effaced']
logical_break = ['other', 'blank', 'ruling']
words['lemma'] = words["cf"] + '[' + words["gw"] + ']' + words["pos"]
words.loc[words["cf"] == "" , 'lemma'] = words['form'] + '[NA]NA'
words.loc[words["pos"] == "n" , 'lemma'] = words['form'] + '[]NU'
words.loc[words["state"].isin(logical_break), 'lemma'] = "break_logical"
words.loc[words["state"].isin(physical_break), 'lemma'] = "break_physical"
words.head(10)

,extent,scope,state,id_word,id_text,form,cf,gw,pos,ftype,label,id_line,lemma
0,n,line,missing,P104232.3,P104232,,,,,,,3,break_physical
1,,,,P104232.5.1,P104232,šu,šu,hand,N,,r 1,5,šu[hand]N
2,,,,P104232.5.2,P104232,ba-ti,teŋ,near,V/i,,r 1,5,teŋ[near]V/i
3,,,,P104232.6.1,P104232,ša₃,šag,heart,N,,r 2,6,šag[heart]N
4,,,,P104232.6.2,P104232,puzur₄-iš-{d}da-gan,Puzrišdagan,1,SN,,r 2,6,Puzrišdagan[1]SN
5,,,,P104232.7.1,P104232,iti,itud,moon,N,,r 3,7,itud[moon]N
6,,,,P104232.7.2,P104232,ki-siki-{d}nin-a-zu,Kisikininazuk,1,MN,,r 3,7,Kisikininazuk[1]MN
7,,,,P104232.7.3,P104232,min-kam,min,two,NU,,r 3,7,min[two]NU
8,,,,P104232.8.1,P104232,mu,mu,year,N,yn,r 4,8,mu[year]N
9,,,,P104232.8.2,P104232,si-mu-ru-um{ki},Si.mu.ru.um,00,SN,yn,r 4,8,Si.mu.ru.um[00]SN


# Proper Nouns in the Ur III Corpus
## The Name Authority

In order to build a network of interactions between individuals in the Treasury corpus we need to be able to extract proper nouns. The lemmatization of proper nouns in this corpus is still in early stages, so that we cannot fully rely on the lemmatized data as represented in our DataFrame. A list of all proper nouns from Drehem texts was put together by John Carnahan and Niek Veldhuis, based on the Drehem texts in [BDTNS](http://bdtns.filol.csic.es/), December 2016. This list includes the form of each name (in transliteration) plus a normalization of that name in [epsd2](http://oracc.org/epsd2) style. The list includes spelling variants as well as names with morphology. Thus, **Ur-{d}en-lil₂**, **Ur-{d}en-lil₂-la₂**, and **Ur-{d}en-lil₂-la₂-ta** are all listed seperately, and are all associated with the same name, as follows: 
```csv
Ur-{d}En-lil₂,UrEnlilak[]PN,
Ur-{d}En-lil₂-la₂,UrEnlilak[]PN,
Ur-{d}En-lil₂-la₂-še₃,UrEnlilak[]PN,
Ur-{d}En-lil₂-la₂-ka-še₃,UrEnlilak[]PN,
Ur-{d}En-lil₂-la₂-ka-ta,UrEnlilak[]PN,
Ur-{d}En-lil₂-la₂-ke₄,UrEnlilak[]PN,
Ur-{d}En-lil₂-la₂-ta,UrEnlilak[]PN,
Ur-{d}En-lil₂-ta,UrEnlilak[]PN,
```
Similarly, name instances that have different spellings but belong to the same person are associated with one normalized form of that name, as in: 
```csv
{d}Šul-gi-si-im-ti,Šulgisimti[]PN, 
{d}Šul-gi-si-im-tum,Šulgisimti[]PN,
```

The list includes all name forms from Drehem (in 2016), currently almost 6,000 entries. 

We will use this list (`drehem_norm_names.csv`, available in the `Normalized` directory) to find proper nouns (Personal Names, Divine Names, Geographical Names, etc.) in our DataFrame and to create more reliable lemmatizations than [epsd2](http://oracc.org/epsd2) can offer today.

In [8]:
normdf = pd.read_csv('Normalized/drehem_norm_names.csv', encoding='utf8')
normdf

,transliteration,normalization,remarks
0,A-AN-ba-az,A.AN.ba.az[]PN,MVN 13 464 r 10 (copy/photo)
1,A-KU-um,A.KU.um[]PN,"Aegyptus 10, 270 27 o 7 (copy)"
2,A-KU.KU-ta,A.KU.KU[]PN,AnOr 12 277 o i 17' (copy)
3,A-NI-ta,A.NI[]PN,Babyl. 7 pl. 22 18 o 3 (copy)
4,A-U.E₂-nu-tuku,A.U.E₂.nu.tuku[]PN,AnOr 07 150 o 2: A-U.KID-nu-tuku IŠ (copy/photo)
...,...,...,...
5802,{d}utu,Utu[]DN,
5803,{d}utu-bar-ra,Utubara[]DN,
5804,Ma₂-gur₈-mah,Magurmah[]ON,
5805,Ma₂-dara₃-abzu,Madaraʾabzu[]ON,


# Reduce Names to Sign Sequences

Many Proper Nouns may validly be transcribed in different ways. Sometimes that is the case because the name is not fully understood or of foreign origin. The city name **a-dam-DUN{ki}** is read variously **a-dam-šah₂{ki}**, **a-dam-dun{ki}**, or **a-dam-DUN{ki}** by different scholars. For a network, the correct reading is of less importance than a consistent reading. All these readings reflect a sign sequence **A DAM DUN KI**, a sequence of signs that cannot plausibly be anything else than a reference to this city.

We will, therefore, use the ORACC Global Sign List ([ogsl](http://oracc.org/ogsl) to transform each name form in our name authority into a sequence of signs. We will do the same with the column `form` of our DataFrame to find matching sign sequences.

Much of the code in the following cells is equivalent to code in Chapter 2.4, where we developed a tool for searching signs in [bdtns](http://bdtns.filol.csic.es/) data.

# Download OGSL

Using code developed in Chapter 2, download the file `ogsl.zip`, which contains all of the [OGSL](http://oracc.org/ogsl) data (sign names and sign readings) in JSON format.

In [9]:
oracc_download(['ogsl']);

Saving http://build-oracc.museum.upenn.edu/json/ogsl.zip as jsonzip/ogsl.zip.


# Sign Equivalencies

In the Ur III period, some signs that are differentiated in [OGSL](http://oracc.org/ogsl) have fallen together. For instance, **DUR₂** and **KU**, which are separate in the Fara period, are represented by the same sign in Ur III. Such signs are listed in the following dictionary. A compiled regular expression is used to replace sign names where appropriate.

In [10]:
equiv = {'ANŠE' : 'GIR₃', 
        'DUR₂' : 'KU', 
        'NAM₂' : 'TUG₂', 
        'TIL' : 'BAD', 
        'NI₂' : 'IM',
        'ŠAR₂' : 'HI', 
        }
w = re.compile(r'\w+') # replace whole words only - do not replace TILLA with BADLA.
           # but do replace |SAL.ANŠE| with |SAL.GIR₃|

# Parse OGSL
The parse function uses the `equiv` dictionary and the compiled regular expression to fill a dictionary, associating each valid sign reading with a sign name.

In [11]:
def parseogsljson(data_json):
    for key, value in data_json["signs"].items():
        key = re.sub(w, lambda m: equiv.get(m.group(), m.group()), key)
        if "values" in value:
            for n in value["values"]:
                d2[n] = key
    return

# Call the OGSL parser
Calling  the function in the previous cell, a dictionary is formed where each `key` is a sign value and each `value` a sign name.

In [12]:
d2 = {}  # this empty dictionary is filled by the parsejson() function, called in this cell.
file = "jsonzip/ogsl.zip"
z = zipfile.ZipFile(file) 
filename = "ogsl/ogsl-sl.json"
signlist = z.read(filename).decode('utf-8')
data_json = json.loads(signlist)                # make it into a json object (essentially a dictionary)
parseogsljson(data_json)  
with open('output/ogsl_dict.p', 'wb') as p:
    pickle.dump(d2, p)  

In [13]:
separators = ['{', '}', '-']
separators2 = ['.', '+', '|']  # used in compound signs
#operators = ['&', '%', '@', '×']
flags = "][?<>⸢⸣⌈⌉*/" # note that ! is omitted from flags, because it is dealt with separately
table = str.maketrans(dict.fromkeys(flags))

In [14]:
def signnames(translit):  
    """This function takes a string of transliterated cuneiform text and translates that string into a string of
    sign names, separated by spaces. In order to work it needs the variables separators, separators2, flags, and table defined above. The variable table
    is used by the translate() method to translate all flags (except for !) to None. The function also needs a dictionary, called d2, that has as
    keys sign readings and sign names as corresponding values. In case a key is not found, the sign reading is replaced by itself."""
    signnames_l = []
    translit = translit.translate(table).lower()  # remove flags, half brackets, square brackets.
    translit = translit.replace('...', 'x')
    for s in separators: # split transliterated form into signs   
        translit = translit.replace(s, ' ').strip()
    s_l = translit.split() # s_l is a list that contains the sequence of transliterated signs without separators or flags
    s_l = [d2.get(sign, sign) for sign in s_l] # replace each transliterated sign with its sign name.
    # Now take care of some special situations: signs with qualifiers, compound signs.
    for sign in s_l:
        if '!' in sign: # corrected sign, as in ka!(SAG), get only the corrected reading.
            sign = sign.split('!(')[0]
            sign = sign.replace('!', '') # remove remaining exclamation marks
        elif sign[-1] == ')' and '(' in sign: # qualified sign, as in ziₓ(SIG₇) - get only the qualifier
            sign = sign.split('(')[1][:-1]
        if '×' in sign: #compound. Compound like |KA×NINDA| to be replaced by |KA×GAR|
            sign_l = sign.replace('|', '').split('×')
            #replace individual signs of the compound by OGSL names
            sign_l = [d2.get(sign, sign) for sign in sign_l] 
            # if user enters |KA*EŠ| this is transformed to ['KA', '|U.U.U|']. The pipes around U.U.U must be replaced by brackets
            sign_l = [f'({sign[1:-1]})' if len(sign) > 1 and sign[0] == '|' else sign for sign in sign_l]
            sign = f"|{'×'.join(sign_l)}|"  #put the sign together again with enclosing pipes.
        elif '.' in sign or '+' in sign: # using elif, so that compounds like |UD×(U.U.U)| are not further analyzed.
            for s in separators2:
                sign = sign.replace(s, ' ').strip() 
            sign_l = sign.split()  # compound sign split into multiple signs
            sign_l = [d2.get(sign, sign) for sign in sign_l]
            for se in separators2:   # in case d2.get returns a compound sign name
                sign_l = [si.replace(se, ' ').strip() for si in sign_l]
            signnames_l.extend(sign_l)
            continue
        sign = d2.get(sign, sign)
        signnames_l.append(sign)
    # add space before and after each line so that each sign representation is enclosed in spaces
    signnames = f" {' '.join(signnames_l).upper()} " 
    return signnames

# Create a Column Sign_Names
Use the function in the cell above to add a new column column, called `sign_names` to  the name authority (the DataFrame `normdf`) and to the `words` DataFrame.

In [15]:
normdf["sign_names"] = normdf["transliteration"].progress_map(signnames)
words['sign_names'] = words['form'].progress_map(signnames)

# Proper Noun Dictionary
Transform the Name Authority in a dictionary with `sign_names` as key and `normalization` as value and use this dictionary to transform words marked as Proper Nouns or 'X' (unknown) in the Part of Speech (`pos`) column into normalized names.

In [16]:
proper_nouns = ['RN', 'PN', 'DN', 'AN', 'WN', 'ON', 'TN', 'CN', 'GN', 'SN']
normd2 = dict(zip(normdf['sign_names'], normdf['normalization']))
words.loc[words.pos.isin(proper_nouns + ['X']), 'lemma'] = words.progress_apply(lambda x: normd2.get(x['sign_names'], x['lemma']), axis=1)

# Apply Corrections
The Ur III data sets as represented in [epsd2/admin/ur3](http://oracc.org/epsd2/admin/ur3) may still contain errors, which will  surface later in the process. For instance, a name does not show up in the analysis, because it has not been lemmatized as a name. Or, the other way around, a word may have been falsely lemmatized as a name. When found, such errors can be corrected (temporarily) in the file `corr.csv` in the directory `Normalized`, associating forms with correct lemmatizations. The final line of code in this cell will correct the column `pos` when necessary.

In [17]:
corrections = pd.read_csv('Normalized/corr.csv', encoding='utf8')
corr_d = dict(zip(corrections['form'], corrections['corr']))
words.loc[words.pos.isin(proper_nouns + ['X']), 'lemma'] = words.progress_apply(lambda x: corr_d.get(x['form'], x['lemma']), axis =1)
words['pos'] = [w.split(']')[-1] if ']' in w else '' for w in words['lemma']]

# Document Typology
Different types of documents will be treated differently. In the Treasure Archive the main document types are Intake, Expenditure, and Transfer.

|  type           | keyword         | lemma(s)              | description |
|----------------|------------------|-----------------------|-------------|
| Intake      | mu-kuₓ(DU)      | mu.DU\[delivery\]N      | materials delivered to the administration |
| Expenditure | ba-zi or zi-ga  | zig[rise]V/i          | materials expended by the administration |
| Transfer    | šu ba-ti        | šu\[hand\]N teŋ\[near\]V/i | materials transfered from one office to another |

In the great majority of cases they canbe identified simply by looking  for the keyword lemmas.

This typology is discussed in much detail in Paoletti's *Der König und sein Kreis* (2012), Chapter 2.1 and 3.1.

In [18]:
texttype = []
for i in ids:
    ttype = ''
    text = list(words.loc[words.id_text == i, 'lemma'])
    if 'mu.DU[delivery]N' in text:
        ttype = 'intake'
    elif 'zig[rise]V/i' in text:
        ttype = 'expenditure'
    elif 'teŋ[near]V/i' in text: 
        ttype = 'transfer'
    texttype.append(ttype)
treasure = dict(zip(ids, texttype))

# Proper Nouns

For the node list, select Personal Names (PN) and Royal Names (RN) but skip those that appear in Year Names or Seals. Add lugal (king) sukkalmah (prime minister) and nin (queen), because these actors may be mentioned without personal name.

In [19]:
unnamed = {'lugal[king]N', 'nin[queen]N', 'sukkalmah[official]N'}
names = set(words.loc[words.pos.isin(['PN', 'RN', 'DN']), 'lemma'])
actors = unnamed | names
seal = list(words.loc[words.label.str.contains('seal'), 'id_word'])
yn = list(words.loc[words.ftype == 'yn', 'id_word'])
exclude = seal + yn
PNs = words.loc[(words.lemma.isin(actors)) & (~words.id_word.isin(exclude))].copy()

# Define keywords and the corresponding roles
The accounts use key words to indicate the particular role of each person in a document. Some of these key words (or key phrases, in some instances) follow the Proper Noun, others precede it. Thus when the word **ŋiri\[foot\]N** immediately precedes a Personal Name, that person is an intermediary (or transporter). A Personal Name followed by **šu\[hand\]N teŋ\[near\]V/i** marks the recipient of goods.

The keywords are associated with a role in two separate dictionaries.

In [20]:
key_post = {'mu.DU[delivery]N': 'deliverer', 'maškim[administrator]N' : 'representative', 
               'zig[rise]V/i' : 'expender',
               'šu[hand]N teŋ[near]V/i': 'recipient' , 'šu[hand]N us[follow]V/t': 'sender'}
key_pre =  {'ŋiri[foot]N' : 'intermediary', 
            'arua[offering]N' : 'offerer', 'kišib[seal]N' : 'sealer', 
            'gabari[copy]N kišib[seal]N' : 'sealer', 'mu.DU[delivery]N' : 'deliverer',
            'mu[name]N' : 'reason', 'ki[place]N' : 'source', 'kiŋ[work]N ak[do]N' : 'producer'}

# Determine the Role of each PN
In order to identify the role of each PN the dictionaries defined in the previous cell are used. Most of the keywords are found on the same line as the PN. 
# TODO: 
default role in texttype transfer. Check default roles for other text types.

In [21]:
role = []
attribute = []
for i in PNs.index:                           # the index of PNs is identical to the one of words
    Pno = PNs.loc[i]['id_text']                    # the text ID (P number)
    lineno = PNs.loc[i]['id_line']                 # the line number inwhich the PN appears
    text = words.loc[words.id_text == Pno]                # the entire text
    line = text.loc[text.id_line == lineno] #  the line with the PN
    mnw = line.index[0]                 # index no of firstword in line
    mxw = line.index[-1]                   # index no. of last word in line
    mx = text.index[-1]                    # index no. of last word in text
    mxl = text.loc[mx]['id_line']          # highest line number in text
    if lineno < mxl:                       # check that nextline is still in the same text
        nextlno = words.loc[mxw+1]['id_line']
        nextline = text.loc[text.id_line == nextlno] # words in nextline
    else:
        nextline = line
    attr = []
    r = ''
    if i < mxw:
        attr = list(line.loc[i+1:]['lemma'])
    if mnw == i:               # PN in first position
        r = key_post.get(line.loc[mxw]['lemma'], r)    # keyword appears in same line in last position
        r = key_post.get(words.loc[mxw+1]['lemma'], r) # or in the next line in first position
        if len(line) > 1:                   # additional words are attributes of the name
            lastwords = ' '.join(list(line[-2:]['lemma'])) # two-word keywords (as in šu ba-ti)
            r = key_post.get(lastwords, r)
        if len(nextline) > 1:               # two-word keyword appearing in first position in next line
            firstwords = ' '.join(list(nextline[:2]['lemma']))
            r = key_post.get(firstwords, r)
            
    elif i > mnw:             # PN appears further in line with keyword(s) preceding
        firstwords = ' '.join(list(line.loc[mnw:i-1]['lemma']))  # join all words before PN
        r = key_pre.get(firstwords, r)        # ŋiri₃ PN; ki PN-ta; kin ak PN; etc
        if line.loc[mnw]['lemma'] == 'ki[place]N' and line.loc[mxw]['form'].endswith('-še₃'): 
            r = 'destination'              # special case: ki PN-še₃         
        if r == '':                # role has not been filled yet
            PN = [w for w in line.loc[mnw:i-1]['lemma'] if w in list(PNs['lemma'])] # is there another PN previously in the line?
            if PN:
                if words.loc[i-1]['lemma'] == 'u[and]CNJ':
                    r = role[-1]               # same role as previous PN
                else:
                    r = 'relation'             # as in PN dumu PN
            elif treasure.get(Pno) == 'expenditure': # commodities + PN: recipient
                r = 'recipient'
            else:                              # if there is no preceding PN: drop the name
                r = 'none'                     # this may  need refinement
        if PNs.loc[i]['lemma'] in unnamed and words.loc[i-1]['lemma'] in list(PNs['lemma']):  
            r = 'none'       # 'unnamed' person is preceded byname
    if r == '' :
        if treasure.get(Pno) == 'expenditure': # default role for zi-ga/ba-zi texts
            r = 'recipient'
        elif treasure.get(Pno) == 'intake' or treasure.get(Pno) == 'transfer':    # default role for mu-DU texts
            r = 'source'
    role.append(r)
    attribute.append(' '.join(attr))
PNs['role'] = role
PNs['attribute'] = attribute

# Show results in a table with links for checking

In [22]:
anchor = '<a href="http://build-oracc.museum.upenn.edu/epsd2/admin/ur3/{}", target="_blank">{}</a>'
PNs2 = PNs.copy()
PNs2['id_word'] = [anchor.format(val,val) for val in PNs['id_word']]

In [23]:
@interact(rows = (1, len(PNs2), 1))
def showpns(rows = 25): 
    return PNs2.loc[PNs2.id_text == 'P103763', ['id_word', 'form', 'pos', 'lemma', 'role', 'attribute']][:rows].style

interactive(children=(IntSlider(value=25, description='rows', max=1270, min=1), Output()), _dom_classes=('widg…

# Create Edges
Needs checking. Deal with roles 'offerer' and 'reason'

In [24]:
intake = [i for i in ids if treasure.get(i) == 'intake']

In [25]:
edges = []
for i in ids:                              # one text ID at a time; muDU texts
    people = PNs.index[PNs.id_text == i]    # indexes of all the people in that text
    for p in people:                       # iterate over those indexes
        source = ''
        target = ''
        role = PNs.loc[p]['role']
        if role in ['intermediary', 'representative']:
            source = PNs.loc[p]['lemma']
            q = [n for n in people if n > p and PNs.loc[n]['role'] in ['recipient', 'sealer']]  # look for target after the source
            if q:
                target = PNs.loc[q[0]]['lemma']
        elif role in ['sender', 'deliverer', 'source']:
            source = PNs.loc[p]['lemma']
            q = [n for n in people if n > p and PNs.loc[n]['role'] in ['recipient', 'sealer', 'intermediary']] 
            if q:
                target = PNs.loc[q[0]]['lemma']
        elif role == 'relation':
            source = PNs.loc[p]['lemma']
            q = [n for n in people if n < p]
            if q:
                target = PNs.loc[q[-1]]['lemma']
        elif role in ['recipient', 'sealer']:
            target = PNs.loc[p]['lemma']
            q = [n for n in people if n > p and PNs.loc[n]['role'] in ['representative', 'source', 'intermediary']]
            if q:
                source = PNs.loc[q[0]]['lemma']
        if source and target:
            edges.append([source, target, i])

In [26]:
edgs = pd.DataFrame(edges)
edgs.columns = ['source', 'target', 'id_text']
anchor = '<a href="http://build-oracc.museum.upenn.edu/epsd2/admin/ur3/{}", target="_blank">{}</a>'
edgs2 = edgs.copy()
edgs2['id_text'] = [anchor.format(val,val) for val in edgs['id_text']]

In [27]:
@interact(rows = (1, len(edgs2), 1))
def showedges(rows = 25): 
    return edgs2.loc[edgs2.duplicated()][:rows].style

interactive(children=(IntSlider(value=25, description='rows', max=263, min=1), Output()), _dom_classes=('widge…

TODO: remove duplicates; make multiple edges between the same people into weights; make into directed graph

In [28]:
edgs = edgs.drop_duplicates()

In [29]:
edgs2 = edgs.groupby(['source', 'target']).agg({'id_text': list}).reset_index()

In [30]:
edgs2['weight'] = [len(i) for i in edgs2['id_text']]
edgs2

,source,target,id_text,weight
0,2[00]PN,2[00]PN,[P127321],1
1,2[00]PN,Amurili[]PN,[P129700],1
2,2[00]PN,Huba[]PN,[P134048],1
3,2[00]PN,Itutuʾa[]PN,[P126620],1
4,2[00]PN,Lugal.x[00]PN,[P141796],1
...,...,...,...,...
213,Šuruškin[]PN,Ludiŋirak[]PN,[P124318],1
214,Šušug[]PN,Ludiŋirak[]PN,[P131604],1
215,ŠuʾEnlil[]PN,Abuṭab[]PN,[P134759],1
216,ŠuʾEnlil[]PN,Dayyanummišar[]PN,[P131578],1


In [31]:
edgs2['id_text'] = [','.join(i)for i in edgs2['id_text']]

In [32]:
edgs2

,source,target,id_text,weight
0,2[00]PN,2[00]PN,P127321,1
1,2[00]PN,Amurili[]PN,P129700,1
2,2[00]PN,Huba[]PN,P134048,1
3,2[00]PN,Itutuʾa[]PN,P126620,1
4,2[00]PN,Lugal.x[00]PN,P141796,1
...,...,...,...,...
213,Šuruškin[]PN,Ludiŋirak[]PN,P124318,1
214,Šušug[]PN,Ludiŋirak[]PN,P131604,1
215,ŠuʾEnlil[]PN,Abuṭab[]PN,P134759,1
216,ŠuʾEnlil[]PN,Dayyanummišar[]PN,P131578,1


In [33]:
import networkx as nx
import matplotlib.pyplot as plt

# Create initial Directed Graph

In [34]:
G=nx.convert_matrix.from_pandas_edgelist(edgs2, source = 'source', target = 'target', edge_attr = ['id_text', 'weight'],
                                        create_using = nx.DiGraph())

In [35]:
node_attr = {name : name for name in G.nodes}
degree = {name : G.degree[name] for name in G.nodes}
node_size = {name : G.degree[name]*3 for name in G.nodes}
edge_size = [G[a][b]['weight']*3 for (a,b) in G.edges]
nx.set_node_attributes(G, node_attr, "name")
nx.set_node_attributes(G, degree, "degree")
nx.set_node_attributes(G, node_size, "node_size")
nx.set_edge_attributes(G, edge_size, "edge_size")

# Import Bokeh

In [36]:
from bokeh.io import output_file, show
from bokeh.models import (BoxSelectTool, Circle, EdgesAndLinkedNodes, HoverTool,
                          MultiLine, NodesAndLinkedEdges, Plot, Range1d, TapTool,
                         BoxZoomTool, ResetTool, OpenURL, CustomJS, Column, SaveTool)
from bokeh.palettes import Spectral4
from bokeh.plotting import figure, output_notebook
from bokeh.models.graphs import from_networkx
from bokeh.models import TextInput, Button
output_notebook()

Loading BokehJS ...

# Tooltips for Nodes

In [ ]:
plot = Plot(plot_width=900, plot_height=900,
            x_range=Range1d(-1.1, 1.1), y_range=Range1d(-1.1, 1.1))
plot.title.text = "Treasure Archive Drehem"

node_hover_tool = HoverTool(tooltips=[("name", "@name"), ("degree", "@degree")])
plot.add_tools(node_hover_tool, BoxZoomTool(), ResetTool())

graph_renderer = from_networkx(G, nx.spring_layout, scale=1, center=(0, 0))

graph_renderer.node_renderer.glyph = Circle(size='node_size', fill_color=Spectral4[0])
graph_renderer.edge_renderer.glyph = MultiLine(line_color=Spectral4[1], line_alpha=0.8, line_width='edge_size')
plot.renderers.append(graph_renderer)

output_file("interactive_graphs.html")
show(plot)

In [ ]:
plot = figure(plot_width=500, plot_height=500,
            x_range=Range1d(-1.1,1.1), y_range=Range1d(-1.1,1.1))

node_hover_tool = HoverTool(tooltips=[("name", "@name"), ("degree", "@degree")])
plot.add_tools(node_hover_tool, BoxZoomTool(), ResetTool())

r = from_networkx(G, nx.circular_layout, scale=1, center=(0,0))

r.node_renderer.glyph = Circle(size='node_size', fill_color='#2b83ba')
r.node_renderer.hover_glyph = Circle(size='node_size', fill_color='#abdda4')

r.edge_renderer.glyph = MultiLine(line_alpha=0, line_width='edge_size')  # zero line alpha
r.edge_renderer.hover_glyph = MultiLine(line_color='#abdda4', line_width=5)

r.inspection_policy = NodesAndLinkedEdges()
plot.renderers.append(r)

show(plot)

In [37]:
# Show with Bokeh
plot = Plot(plot_width=1000, plot_height=900,
            x_range=Range1d(-2, 2), y_range=Range1d(-2, 2))
plot.title.text = "Drehem Treasure Archive"

graph_renderer = from_networkx(G, nx.circular_layout, scale=1.9, center=(0, 0))
#graph_renderer.edge_renderer.data_source.data["line_width"] = [G.get_edge_data(a,b)['weight'] for a, b in G.edges()]
#graph_renderer.edge_renderer.glyph.line_width = {'field': 'line_width'}
graph_renderer.edge_renderer.glyph.line_alpha = 0.8
graph_renderer.node_renderer.selection_glyph = Circle(size='degree', fill_color=Spectral4[2])
graph_renderer.node_renderer.hover_glyph = Circle(size='degree', fill_color=Spectral4[1])
graph_renderer.node_renderer.glyph = Circle(size='degree', fill_color=Spectral4[0])
graph_renderer.edge_renderer.glyph = MultiLine(line_color="#CCCCCC", line_alpha=0.8, line_width = 'weight')
graph_renderer.edge_renderer.selection_glyph = MultiLine(line_color=Spectral4[2], line_width = 'weight')
graph_renderer.edge_renderer.hover_glyph = MultiLine(line_color=Spectral4[1], line_width =  'weight')

graph_renderer.inspection_policy = NodesAndLinkedEdges()
graph_renderer.selection_policy = EdgesAndLinkedNodes()

node_hover_tool = HoverTool(tooltips=[("name", "@name"), ('degree', '@value')])
plot.renderers.append(graph_renderer)
plot.add_tools(node_hover_tool, BoxZoomTool(), ResetTool())
output_file("vis/interactive_graphs.html")
show(plot)

# Tooltips for Edges
The following code draws a network of the Drehem Treasure Archive. By selecting a node (click on the node) the node and all its direct neighbors are highlighted and the button below the drawing links to the collection of texts in ORACC from which these edges come.
# TODO
The list of P numbers is currently not unique - a P number may provide more than one relevant edge.

In [ ]:
plot = Plot(plot_width = 900, plot_height = 900,
            x_range = Range1d(-1.1, 1.1), y_range = Range1d(-1.1, 1.1))
plot.title.text = "Drehem Treasure Archive"

plot.add_tools(HoverTool(tooltips = [('Text ID', '@id_text'), ('start', '@start'), ('end', '@end')]), 
               TapTool(), BoxSelectTool(), ResetTool(), BoxZoomTool(), SaveTool())

graph_renderer = from_networkx(G, nx.circular_layout, scale = 1, center = (0, 0))

graph_renderer.node_renderer.glyph = Circle(size = 'node_size', fill_color = Spectral4[0])
graph_renderer.node_renderer.selection_glyph = Circle(size = 'node_size', fill_color = Spectral4[2])
graph_renderer.node_renderer.hover_glyph = Circle(size = 'node_size', fill_color = Spectral4[1])

graph_renderer.edge_renderer.glyph = MultiLine(line_color = "#CCCCCC", line_alpha = 0.8, line_width = 5)
graph_renderer.edge_renderer.selection_glyph = MultiLine(line_color = Spectral4[2], line_width = 5)
graph_renderer.edge_renderer.hover_glyph = MultiLine(line_color = Spectral4[1], line_width = 5)

graph_renderer.selection_policy = NodesAndLinkedEdges()
graph_renderer.inspection_policy = EdgesAndLinkedNodes()

plot.renderers.append(graph_renderer)

info_text_ids = TextInput(title = 'Text IDs:', value = '')
info_start_end = TextInput(title = 'Start => End Values:', value = '')
url = "http://oracc.org/epsd2/admin/ur3/"
esource = graph_renderer.edge_renderer.data_source
code = """  start_end_values = []
            text_ids = []
            for(idx in esource.selected.indices)
            {
                index = esource.selected.indices[idx]
                start_end_values.push(esource.data['start'][index] + '=>' + esource.data['end'][index] ); 
                text_ids.push(esource.data['id_text'][index]) 
            }
            info_start_end.value = String(start_end_values);
            info_text_ids.value = String(text_ids);"""
code2 = """ urlnew = url.concat(info_text_ids.value);
            window.open(urlnew)"""
callback = CustomJS(args = dict(esource = esource, 
                                info_text_ids = info_text_ids, 
                                info_start_end = info_start_end), 
                    code = code)
plot.select_one(TapTool).callback = callback
button = Button(label="Click to open text editions", button_type="success")
button.js_on_click(CustomJS(args = dict(url=url, info_text_ids=info_text_ids), 
                        code=code2))
show(Column(plot, info_start_end, info_text_ids, button))

In [38]:
plot = Plot(plot_width = 900, plot_height = 900,
            x_range = Range1d(-1.1, 1.1), y_range = Range1d(-1.1, 1.1))
plot.title.text = "Drehem Treasure Archive"

plot.add_tools(HoverTool(tooltips = [('Text ID', '@id_text'), ('start', '@start'), ('end', '@end')]), 
               TapTool(), BoxSelectTool(), ResetTool(), BoxZoomTool(), SaveTool())

graph_renderer = from_networkx(G, nx.circular_layout, scale = 1, center = (0, 0))

graph_renderer.node_renderer.glyph = Circle(size = 'node_size', fill_color = Spectral4[0])
graph_renderer.node_renderer.selection_glyph = Circle(size = 'node_size', fill_color = Spectral4[2])
graph_renderer.node_renderer.hover_glyph = Circle(size = 'node_size', fill_color = Spectral4[1])

graph_renderer.edge_renderer.glyph = MultiLine(line_color = "#CCCCCC", line_alpha = 0.8, line_width = 5)
graph_renderer.edge_renderer.selection_glyph = MultiLine(line_color = Spectral4[2], line_width = 5)
graph_renderer.edge_renderer.hover_glyph = MultiLine(line_color = Spectral4[1], line_width = 5)

graph_renderer.selection_policy = NodesAndLinkedEdges()
graph_renderer.inspection_policy = EdgesAndLinkedNodes()

plot.renderers.append(graph_renderer)

info_text_ids = TextInput(title = 'Text IDs:', value = '')
info_start_end = TextInput(title = 'Start => End Values:', value = '')
url = "http://oracc.org/epsd2/admin/ur3/"
esource = graph_renderer.edge_renderer.data_source
code = """  start_end_values = []
            text_ids = []
            for(idx in esource.selected.indices)
            {
                index = esource.selected.indices[idx]
                start_end_values.push(esource.data['start'][index] + '=>' + esource.data['end'][index] ); 
                text_ids.push(esource.data['id_text'][index]) 
            }
            info_start_end.value = String(start_end_values);
            info_text_ids.value = String(text_ids);"""
code2 = """ urlnew = url.concat(info_text_ids.value);
            window.open(urlnew)"""
callback = CustomJS(args = dict(esource = esource, 
                                info_text_ids = info_text_ids, 
                                info_start_end = info_start_end), 
                    code = code)
plot.select_one(TapTool).callback = callback
button = Button(label="Click to open text editions", button_type="success")
button.js_on_click(CustomJS(args = dict(url=url, info_text_ids=info_text_ids), 
                        code=code2))
show(Column(plot, info_start_end, info_text_ids, button))